In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pandas as pd
import seaborn as sns
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
import numpy as np
import pandas as pd
import re

In [3]:
train_set = pd.read_csv('data/train.csv')
test_set = pd.read_csv('data/test.csv')
sub_set = pd.read_csv('data/sample_submit.csv')

In [15]:
train_set


,ID,Gender,Age,Customer Type,Type of Travel,Class,Flight Distance,Departure Delay,Arrival Delay,Departure and Arrival Time Convenience,...,On-board Service,Seat Comfort,Leg Room Service,Cleanliness,Food and Drink,In-flight Service,In-flight Wifi Service,In-flight Entertainment,Baggage Handling,Satisfaction
0,37576,Male,46,Returning,Business,Business,1608,0,0.0,1,...,5,5,5,3,2,5,1,5,5,Satisfied
1,56913,Male,30,Returning,Business,Business,1379,30,32.0,5,...,3,1,5,1,1,3,1,1,3,Neutral or Dissatisfied
2,6541,Female,50,Returning,Business,Business,414,0,0.0,5,...,4,5,4,5,3,4,5,4,4,Satisfied
3,40646,Female,48,Returning,Business,Economy,95,0,0.0,3,...,5,1,5,2,1,5,5,5,5,Satisfied
4,29947,Female,24,Returning,Business,Economy Plus,143,0,0.0,5,...,5,1,2,1,1,3,0,1,5,Satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84962,53730,Male,49,First-time,Business,Business,1310,0,0.0,3,...,4,3,2,3,3,4,4,3,4,Neutral or Dissatisfied
84963,82113,Male,7,Returning,Personal,Economy Plus,650,0,0.0,3,...,2,1,4,1,1,4,2,1,3,Neutral or Dissatisfied
84964,54411,Female,19,Returning,Personal,Economy,363,0,0.0,5,...,5,5,3,5,5,5,3,5,5,Neutral or Dissatisfied
84965,15247,Male,58,Returning,Business,Business,3445,9,5.0,1,...,5,5,5,3,5,5,1,5,5,Satisfied


In [19]:
train_set['Satisfaction'].unique()

array(['Satisfied', 'Neutral or Dissatisfied'], dtype=object)

In [4]:
print(train_set.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84967 entries, 0 to 84966
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   ID                                      84967 non-null  int64  
 1   Gender                                  84967 non-null  object 
 2   Age                                     84967 non-null  int64  
 3   Customer Type                           84967 non-null  object 
 4   Type of Travel                          84967 non-null  object 
 5   Class                                   84967 non-null  object 
 6   Flight Distance                         84967 non-null  int64  
 7   Departure Delay                         84967 non-null  int64  
 8   Arrival Delay                           84705 non-null  float64
 9   Departure and Arrival Time Convenience  84967 non-null  int64  
 10  Ease of Online Booking                  84967 non-null  in

In [5]:
print(test_set.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44913 entries, 0 to 44912
Data columns (total 23 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   ID                                      44913 non-null  int64  
 1   Gender                                  44913 non-null  object 
 2   Age                                     44913 non-null  int64  
 3   Customer Type                           44913 non-null  object 
 4   Type of Travel                          44913 non-null  object 
 5   Class                                   44913 non-null  object 
 6   Flight Distance                         44913 non-null  int64  
 7   Departure Delay                         44913 non-null  int64  
 8   Arrival Delay                           44782 non-null  float64
 9   Departure and Arrival Time Convenience  44913 non-null  int64  
 10  Ease of Online Booking                  44913 non-null  in

In [6]:
def top_columns_with_nulls(dataframe, top_n=5):
    total_rows = len(dataframe)
    null_counts = dataframe.isnull().sum()
    null_percentage = (null_counts / total_rows) * 100
    null_info_df = pd.DataFrame({
        'Column': null_counts.index,
        'Null Count': null_counts.values,
        'Null Percentage': null_percentage.values
    })
    sorted_nulls = null_info_df.sort_values(by='Null Count', ascending=False)
    return sorted_nulls.head(top_n)

print(top_columns_with_nulls(test_set))
print()
print(top_columns_with_nulls(train_set))

           Column  Null Count  Null Percentage
8   Arrival Delay         131         0.291675
1          Gender           0         0.000000
0              ID           0         0.000000
3   Customer Type           0         0.000000
4  Type of Travel           0         0.000000

           Column  Null Count  Null Percentage
8   Arrival Delay         262         0.308355
0              ID           0         0.000000
2             Age           0         0.000000
1          Gender           0         0.000000
4  Type of Travel           0         0.000000


In [7]:
def delete_null_rows(dataframe):
    cleaned_df = dataframe.dropna()
    print(f"Rows removed: {len(dataframe) - len(cleaned_df)}")
    return cleaned_df
def auto_fill_nulls(dataframe):
    filled_df = dataframe.copy()
    for column in filled_df.columns:
        if filled_df[column].isnull().sum() > 0:  
            if np.issubdtype(filled_df[column].dtype, np.number):
                median_value = filled_df[column].median()
                filled_df[column].fillna(median_value, inplace=True)
                print(f"Filled nulls in numeric column '{column}' with median value: {median_value}")
            else:
                top_3_values = filled_df[column].value_counts().index[:3]
                if len(top_3_values) > 0:  
                    random_choice = np.random.choice(top_3_values)
                    filled_df[column].fillna(random_choice, inplace=True)
                    print(f"Filled nulls in non-numeric column '{column}' with random choice from top 3: {random_choice}")
                else:
                    print(f"Column '{column}' has no valid non-null values to use for filling.")
    return filled_df

In [54]:
def classify_gender(df):
    if 'Gender' not in df.columns:
        return df
    df = df.copy()
    df['Gender'] = df['Gender'].str.lower()
    df['Gender'] = df['Gender'].map({'male': 1, 'female': 2}).fillna(3).astype(int)
    return df

def classify_customer_type(df):
    if 'Customer Type' not in df.columns:
        return df
    df = df.copy()
    df['Customer Type'] = df['Customer Type'].str.lower()
    df['Customer Type'] = df['Customer Type'].map({'Returning': 1, 'First-time': 2}).fillna(0).astype(int)
    return df

def classify_type_of_travel(df):
    if 'Type of Travel' not in df.columns:
        return df
    df = df.copy()
    df['Type of Travel'] = df['Type of Travel'].str.lower()
    df['Type of Travel'] = df['Type of Travel'].map({'Business': 1, 'Personal': 2}).fillna(0).astype(int)
    return df

def classify_class(df):
    if 'Class' not in df.columns:
        return df
    df = df.copy()
    df['Class'] = df['Class'].str.lower()
    df['Class'] = df['Class'].map({'Business': 1, 'Economy': 2, 'Economy Plus': 3}).fillna(0).astype(int)
    return df

def classify_satisfaction(df):
    if 'Satisfaction' not in df.columns:
        return df
    df = df.copy()
    df['Satisfaction'] = df['Satisfaction'].str.lower()
    df['Satisfaction'] = df['Satisfaction'].map({'Satisfied': 1, 'Neutral or Dissatisfied': 0}).fillna(-1).astype(int)
    return df

In [55]:
pipeline = Pipeline([
    ('classify_gender', FunctionTransformer(classify_gender, validate=False)),
    ('classify_customer_type', FunctionTransformer(classify_customer_type, validate=False)),
    ('classify_type_of_travel', FunctionTransformer(classify_type_of_travel, validate=False)),
    ('classify_class', FunctionTransformer(classify_class, validate=False)),
    ('classify_satisfaction', FunctionTransformer(classify_satisfaction, validate=False)),
    ('scaler', StandardScaler())
])



In [29]:
def experiment(df):
    return df

In [84]:


def get_data():
    train_set = pd.read_csv('data/train.csv').drop(columns=['ID', 'Arrival Delay'])
    test_set = pd.read_csv('data/test.csv').drop(columns=['ID', 'Arrival Delay'])
    sub_set = pd.read_csv('data/sample_submit.csv')
    # test_set = auto_fill_nulls(test_set).drop(columns=['laptop_ID','TypeName'])
    # test_set = delete_null_rows(test_set)
    # train_set =  auto_fill_nulls(train_set).drop(columns=['laptop_ID', 'TypeName'])

    # test_set, train_set = experiment(test_set), experiment(train_set)

    # train_set = delete_null_rows(train_set)

    # train_set.drop(columns=['TypeName'], inplace=True)
    # test_set.drop(columns=['TypeName'], inplace=True)

    X_train = train_set.drop('Satisfaction', axis=1)
    y_train = train_set['Satisfaction']

    X_test = test_set.copy() 

    X_train = pipeline.fit_transform(X_train)
    X_test = pipeline.transform(test_set)
    return X_train, X_test, y_train

X_train, X_test, y_train = get_data()
print(len(X_test))

44913


In [65]:
"""

filled_test = auto_fill_nulls(test_set)
dropped_test = delete_null_rows(test_set)
filled_train =  auto_fill_nulls(train_set)
dropped_train = delete_null_rows(train_set)
"""
X_test[0]


array([ 0.98961458,  0.40584681,  0.        ,  0.        ,  0.        ,
       -0.30212156, -0.37423223, -0.0201581 ,  0.10604058, -1.94174554,
       -0.36568228, -1.51036359,  0.37319806,  1.0831912 ,  0.3890324 ,
        1.22556154,  1.29288546, -2.36430439,  0.09412778,  1.12418337,
       -1.48648243])

In [66]:

final_feature_names = [
    'Ram', 
    'Weight', 
    'Gpu', 
    'Cpu', 
    'ScreenResolution', 
    'Total_HDD_GB', 
    'Total_SSD_GB', 
    'Total_Flash_GB', 
    'Company', 
    'Product', 'OpSys', 'OpSys', 'OpSys', 'OpSys', 'OpSys', 'OpSys', 'OpSys', 'OpSys', 'OpSys',
     'OpSys', 'OpSys', 'OpSys',
]
data_df = pd.DataFrame(X_train, columns=final_feature_names)

#plt.show()
data_df.info()

ValueError: Shape of passed values is (84967, 21), indices imply (84967, 22)

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split
import random
rs = random.randint(1,1000)
print(f"random state = {rs}")
X_train, X_test, y_train = get_data()

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=rs)

model = LogisticRegression(max_iter=1000)
model.fit(X_tr, y_tr)

# Evaluate the model
y_pred = model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))
print(f"f1: {f1_score(y_val, y_pred, pos_label='Satisfied')}")

random state = 438
Accuracy: 0.8114628692479698
Confusion Matrix:
 [[4799 1809]
 [1395 8991]]
Classification Report:
                          precision    recall  f1-score   support

Neutral or Dissatisfied       0.77      0.73      0.75      6608
              Satisfied       0.83      0.87      0.85     10386

               accuracy                           0.81     16994
              macro avg       0.80      0.80      0.80     16994
           weighted avg       0.81      0.81      0.81     16994

f1: 0.848768054375531


In [82]:
from tqdm import tqdm
def brute_force():
    X_train, X_test, y_train = get_data()
    states_map = {}
    for i in tqdm(range(1000), desc="AXAXAXAXXAXAXA"):
        rs = i
        

        X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=rs)

        model = LogisticRegression(max_iter=100)
        model.fit(X_tr, y_tr)

        # Evaluate the model
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, pos_label='Satisfied')
        states_map[f1] = i
    
    x = max([p for p in states_map])
    return states_map[x], x

#rs, r2 = brute_force()
#print(rs, r2) #386 on 1000 iters

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=386)

model = LogisticRegression(max_iter=100)
model.fit(X_tr, y_tr)

test_predictions = model.predict(X_test)
len([0,1])

2

In [89]:
expected_length = 44915 - 2
actual_length = len(test_predictions)

if actual_length != expected_length:
    raise ValueError(f"Expected {expected_length} predictions, but got {actual_length}.")

mapsss = {
    "Neutral or Dissatisfied": 0,
    "Satisfied": 1
}

submission_df = pd.DataFrame({
    'ID': sub_set['ID'],
    'Prediction':  pd.Series(test_predictions).map({'Neutral or Dissatisfied': 0, 'Satisfied': 1}).astype(int)
})

print(submission_df.head())
print(submission_df.tail())

submission_df.to_csv("SUB.csv", index=False)

print(f"Submission file 'SUB.csv' has been created successfully.")


AttributeError: 'numpy.ndarray' object has no attribute 'map'